# Automated ML

In this notebook, I will perform the following steps:

Import Dependencies
Create an Experiment in an existing Workspace.
Create or Attach existing AmlCompute to a workspace.
Define data loading in a TabularDataset.
Configure AutoML using AutoMLConfig.
Use AutoMLStep
Train the model using AmlCompute
Explore the results.
Test the best fitted model.

## Import Dependcies

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [24]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources
import json
import pickle

import azureml.core
from azureml.core import Model
from azureml.core.environment import Environment
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep
from azureml.widgets import RunDetails
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.model import Model
from azureml.core.webservice import LocalWebservice
from azureml.pipeline.core import PipelineData, TrainingOutput
from azureml.core.model import Model

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

import urllib.request


import joblib



SDK version: 1.39.0


## Create an Azure ML experiment

In [25]:
ws = Workspace.from_config()

# Choose a name for the run history container in the workspace.
experiment_name = 'ml-experiment-1'
project_folder = './automl-project'

experiment = Experiment(ws, experiment_name)
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-192174
aml-quickstarts-192174
southcentralus
2c48c51c-bd47-40d4-abbe-fb8eabd19c8c


## Create or Attach an AmlCompute cluster

In [6]:
# Choose a name for your CPU cluster
amlcompute_cluster_name = "automl-cluster"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)

Found existing cluster, use it.
Succeeded......................................................................................................................
AmlCompute wait for completion finished

Wait timeout has been reached
Current provisioning state of AmlCompute is "Succeeded" and current node count is "0"


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [26]:
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "Heart_Prediction_Dataset"
description_text = "Heart Prediction Dataset"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/camille-wilkens/AZ_Capstone/main/heart_failure_clinical_records_dataset.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00
mean,60.83,0.43,581.84,0.42,38.08,0.35,263358.03,1.39,136.63,0.65,0.32,130.26,0.32
std,11.89,0.50,970.29,0.49,11.83,0.48,97804.24,1.03,4.41,0.48,0.47,77.61,0.47
min,40.00,0.00,23.00,0.00,14.00,0.00,25100.00,0.50,113.00,0.00,0.00,4.00,0.00
25%,51.00,0.00,116.50,0.00,30.00,0.00,212500.00,0.90,134.00,0.00,0.00,73.00,0.00
50%,60.00,0.00,250.00,0.00,38.00,0.00,262000.00,1.10,137.00,1.00,0.00,115.00,0.00
75%,70.00,1.00,582.00,1.00,45.00,1.00,303500.00,1.40,140.00,1.00,1.00,203.00,1.00
max,95.00,1.00,7861.00,1.00,80.00,1.00,850000.00,9.40,148.00,1.00,1.00,285.00,1.00


## Review the Dataset Result

In [27]:
dataset.take(5).to_pandas_dataframe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.00,0,582,0,20,1,265000.00,1.90,130,1,0,4,1
1,55.00,0,7861,0,38,0,263358.03,1.10,136,1,0,6,1
2,65.00,0,146,0,20,0,162000.00,1.30,129,1,1,7,1
3,50.00,1,111,0,20,0,210000.00,1.90,137,1,0,7,1
4,65.00,1,160,1,20,0,327000.00,2.70,116,0,0,8,1


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [28]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes": 30,
    "max_concurrent_iterations": 5,
     "n_cross_validations": 25,
     "primary_metric" : 'AUC_weighted'
}
 #   "primary_metric" : 'AUC_weighted'
# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",   
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [29]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)


Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
ml-experiment-1,AutoML_4cde3cab-81ac-44bf-b019-7a1ebec0790a,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


In [30]:

RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [31]:

remote_run.wait_for_completion(show_output=True)

Experiment,Id,Type,Status,Details Page,Docs Page
ml-experiment-1,AutoML_4cde3cab-81ac-44bf-b019-7a1ebec0790a,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

**********************************************************************************

   47                                                  0:00:00                nan    0.9165
   48    VotingEnsemble                                0:00:04             0.9270    0.9270
   49    StackEnsemble                                 0:00:04             0.9170    0.9270


{'runId': 'AutoML_4cde3cab-81ac-44bf-b019-7a1ebec0790a',
 'target': 'automl-cluster',
 'status': 'Completed',
 'startTimeUtc': '2022-04-15T18:09:41.612963Z',
 'endTimeUtc': '2022-04-15T18:32:43.439137Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 20 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '25',
  'target': 'automl-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"ml-experiment-1","subscription_id":"2c48c51c-bd47-40d4-abbe-fb8eabd19c8c","resource_group":"aml-quickstarts-192174","workspace_name":"quick-starts-ws-192174","region":"southcentralus","compute_target":"aut

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [32]:
best_run,fitted_model = remote_run.get_output()
best_run_metrics = best_run.get_metrics() 

print("Best Run:",best_run.id)
print(best_run)
print("Fitted Model:", fitted_model)
print("Fitted Model Steps:")
print(fitted_model.steps)
print("Best Run Metrics")
print(best_run_metrics)

for i in best_run_metrics:
    x = best_run_metrics[i]
    print(i,x )


Package:azureml-automl-runtime, training version:1.40.0, current version:1.39.0
Package:azureml-core, training version:1.40.0, current version:1.39.0
Package:azureml-dataprep, training version:3.0.1, current version:2.27.1
Package:azureml-dataprep-rslex, training version:2.4.1, current version:2.3.1
Package:azureml-dataset-runtime, training version:1.40.0, current version:1.39.0
Package:azureml-defaults, training version:1.40.0, current version:1.39.0
Package:azureml-interpret, training version:1.40.0, current version:1.39.0
Package:azureml-mlflow, training version:1.40.0, current version:1.39.0
Package:azureml-pipeline-core, training version:1.40.0, current version:1.39.0
Package:azureml-responsibleai, training version:1.40.0, current version:1.39.0
Package:azureml-telemetry, training version:1.40.0, current version:1.39.0
Package:azureml-train-automl-client, training version:1.40.0, current version:1.39.0
Package:azureml-train-automl-runtime, training version:1.40.0.post1, current ve

Best Run: AutoML_4cde3cab-81ac-44bf-b019-7a1ebec0790a_48
Run(Experiment: ml-experiment-1,
Id: AutoML_4cde3cab-81ac-44bf-b019-7a1ebec0790a_48,
Type: azureml.scriptrun,
Status: Completed)
Fitted Model: Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('30', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=0.8, eta=0.1, grow_policy='lossguide', max_bin=255, max_depth=3, max_leaves=7, n_estimators=100, n_jobs=1, 

In [64]:
print(fitted_model.steps)

[('datatransformer', DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, task='classification')), ('prefittedsoftvotingclassifier', PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('30', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=0.8, eta=0.1, grow_policy='lossguide', max_bin=255, max_depth=3, max_leaves=7, n_estimators=100, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=0, reg_alpha=0, reg_lambda=0.10416666666666667, subsample=0.6, tree_method='hist'))], verbose=False)), ('41', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWra

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

### Register Model

In [58]:
#Steps Followed from: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-deploy-and-where?tabs=azcli

from azureml.core import Model

automl_model = remote_run.register_model(model_name='Heart_Failure_Prediction_Model')
automl_model.download(target_dir="outputs", exist_ok = True)

#model = Model(ws,'Heart_Failure_Prediction_Model')

'outputs/model.pkl'

In [57]:
for model in Model.list(ws):
    print(model.name,model.version)

Heart_Failure_Prediction_Modela 1
Heart_Failure_Prediction_Model 6
Heart_Failure_Prediction_Model 5
Heart_Failure_Prediction_Model 4
Heart_Failure_Prediction_Model 3
Heart_Failure_Prediction_Model 2


### Create Inference Config

In [61]:

from azureml.automl.core.shared import constants
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig

myenv = Environment.from_conda_specification(name="env", file_path="project_environment.yml")
inference_config = InferenceConfig(entry_script="score.py",environment=myenv)

TODO: In the cell below, print the logs of the web service and delete the service

### Deploy Webservice

In [62]:
from azureml.core.webservice import Webservice, AciWebservice
model = Model(ws,'Heart_Failure_Prediction_Model')

deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb = 2, 
                                                       description='Heart_Failure_Prediction_Model',enable_app_insights=True )


service=Model.deploy(workspace=ws,
                    name="heart-failure-service",
                    models=[model],
                    inference_config=inference_config,
                    deployment_config=deployment_config)

service.wait_for_deployment(show_output=True)





Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2022-04-15 19:40:37+00:00 Creating Container Registry if not exists.
2022-04-15 19:40:37+00:00 Registering the environment.
2022-04-15 19:40:38+00:00 Use the existing image.
2022-04-15 19:40:38+00:00 Generating deployment configuration.
2022-04-15 19:40:39+00:00 Submitting deployment to compute.
2022-04-15 19:40:42+00:00 Checking the status of deployment heart-failure-service-d..
2022-04-15 19:43:17+00:00 Checking the status of inference endpoint heart-failure-service-d.
Failed


ERROR:azureml.core.webservice.webservice:Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: d4a42db2-ab4d-4eb9-a380-83ebfd05f92b
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Error in entry script, AttributeError: module 'xgboost.compat' has no attribute 'XGBoostLabelEncoder', please run print(service.get_logs()) to get details.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, AttributeError: module 'xgboost.compat' has no attribute 'XGBoostLabelEncoder', please run print(service.get_logs()) to get details."
    }
  ]
}



WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: d4a42db2-ab4d-4eb9-a380-83ebfd05f92b
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Error in entry script, AttributeError: module 'xgboost.compat' has no attribute 'XGBoostLabelEncoder', please run print(service.get_logs()) to get details.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, AttributeError: module 'xgboost.compat' has no attribute 'XGBoostLabelEncoder', please run print(service.get_logs()) to get details."
    }
  ]
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached non-successful terminal state, current service state: Failed\nOperation ID: d4a42db2-ab4d-4eb9-a380-83ebfd05f92b\nMore informat

In [54]:
print(service.get_logs())

2022-04-15T19:15:54,246674100+00:00 - gunicorn/run 
Dynamic Python package installation is disabled.
Starting HTTP server
2022-04-15T19:15:54,257013900+00:00 - rsyslog/run 
2022-04-15T19:15:54,268594800+00:00 - nginx/run 
2022-04-15T19:15:54,265849500+00:00 - iot-server/run 
rsyslogd: /azureml-envs/azureml_fc7e1840225416a471556263041ca33e/lib/libuuid.so.1: no version information available (required by rsyslogd)
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2022-04-15T19:15:54,661630600+00:00 - iot-server/finish 1 0
2022-04-15T19:15:54,668077000+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 19.9.0
Listening at: http://127.0.0.1:31311 (379)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 407
SPARK_HOME not set. Skipping PySpark Initialization.
Generating new fontManager, this may take some time...
Initializing logger
2022-04-15 19:16:00,108 | root | INFO | Starting up app insights client
logging socket wa

### Test Webservice

In [ ]:
import json
import requests


test = {
  "data": [ {
      "age": 90,
      "anaemia": 0,
      "creatinine_phosphokinase": 500, 
      "diabetes":  1,
      "ejection_fraction": 150,
      "high_blood_pressure": 0,
      "platelets": 100000,
      "serum_creatinine": 2.75,
      "serum_sodium": 140,
      "sex": 1,
      "smoking": 0,
      "time": 500 },
      
      {"age": 45,
      "anaemia": 1,
      "creatinine_phosphokinase": 500, 
      "diabetes":  1,
      "ejection_fraction": 50,
      "high_blood_pressure": 1,
      "platelets": 100000,
      "serum_creatinine": 1.75,
      "serum_sodium": 140,
      "sex": 0,
      "smoking": 1,
      "time": 213}
  ]
}


test_data = json.dumps(test)

# Make the request and display the response
response = requests.post(service.scoring_uri, test_data, headers={'Content-Type': 'application/json'})


print(response.json())


### Delete Webservice

In [22]:
service.delete()

model.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
